# Cours: Dualité Rotation/Translation en 6x6 "tout rotations"
## Espace (X) + Temps 3D (Y), sondes, et vitesse bornée par c

**Idée clé :** on modélise en 6x6 les transformations locales :
- X ∈ R^3 : triade spatiale ("espace")
- Y ∈ R^3 : triade temporelle ("temps 3D")
- M = [[R_ext, E_ext],[E_int, R_int]], chaque bloc 3x3 ∈ SO(3)
  - R_ext : rotation de la triade spatiale
  - R_int : rotation de la triade temporelle (orientation d'horloge)
  - E_ext : "couplage temps→espace" (lecture **translation** via rotations)
  - E_int : "couplage espace→temps" (rétroaction sur l'horloge)
- S = [[SXX, SXY],[SYX, SYY]] ∈ SO(3)×4 : état-opérateur (champ) préparant (X_eff, Y_eff)

**Sondes :**
- Sonde droite (préparation) f_{i,j} = [e_i ; e_j] (6x1) — choisit deux axes canoniques
- Sonde gauche scalaire ℓ_{p,q}^T = [e_p^T  e_q^T] (1x6) — lit une composante de X' et/ou Y'

**Lecture espace-temps (par multiplications uniquement) :**
- V0 = S f, V1 = M V0, avec V1 = [X' ; Y']
- Δx = e_s (R_ext X_eff + E_ext Y_eff)
- Δτ_vec = e_t (E_int X_eff + R_int Y_eff), Δτ = u_t^T Δτ_vec
- v = ||Δx|| / Δτ
où (e_s, e_t) sont des vielbeins (métriques locales) et u_t est l'axe d'horloge lu.

**Objectifs pédagogiques :**
- Mettre la translation sur le même pied que la rotation (via E_ext ∈ SO(3))
- Conserver l’esthétique "tout rotations" (pas de colonne affine)
- Imposer v ≤ c par choix de pas (Δt) et par design de S (pure rotation)

In [1]:
import sympy as sp
import pandas as pd
from IPython.display import display

sp.init_printing()
I3 = sp.eye(3)

# --- Rotations élémentaires + Euler ---
def Rx(a):
    c, s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])

def Ry(a):
    c, s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])

def Rz(a):
    c, s = sp.cos(a), sp.sin(a)
    return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])

def euler_R(roll, pitch, yaw, order="ZYX"):
    order = order.upper()
    if order == "ZYX":
        return Rz(yaw) * Ry(pitch) * Rx(roll)
    if order == "XYZ":
        return Rx(roll) * Ry(pitch) * Rz(yaw)
    raise ValueError("ordre = 'ZYX' ou 'XYZ'")

def ok_SO3(R, tol=1e-10):
    RtR = (R.T * R).evalf()
    det = sp.N(R.det())
    return (RtR - sp.eye(3)).norm() < tol and abs(det - 1) < tol

# --- Utilitaires géométriques (pour "designer" des pas) ---
def rotation_from_two_vectors(a, b):
    """
    R ∈ SO(3) tel que R a = b, pour a,b unitaires (gère a = +/- b).
    """
    a = sp.Matrix(a); b = sp.Matrix(b)
    if (a - b).norm() < 1e-12:
        return I3
    if (a + b).norm() < 1e-12:
        # 180° autour d'un axe orthogonal à a
        if abs(a[0]) < 0.9:
            axis = sp.Matrix([1,0,0]).cross(a)
        else:
            axis = sp.Matrix([0,1,0]).cross(a)
        axis = axis / axis.norm()
        vx, vy, vz = axis
        Vx = sp.Matrix([[0,-vz,vy],[vz,0,-vx],[-vy,vx,0]])
        return I3 + 2 * Vx * Vx
    v = a.cross(b)
    s = v.norm()
    c = (a.T * b)[0]
    v = v / s
    vx, vy, vz = v
    Vx = sp.Matrix([[0,-vz,vy],[vz,0,-vx],[-vy,vx,0]])
    ang = sp.acos(c)
    return I3 + sp.sin(ang) * Vx + (1 - sp.cos(ang)) * (Vx * Vx)

def any_perp(u):
    """
    Retourne unitaire n perpendiculaire à u (u non nul).
    """
    u = sp.Matrix(u)
    trial = sp.Matrix([1,0,0]) if abs(u[0]) < 0.9 else sp.Matrix([0,1,0])
    n = u.cross(trial)
    n = n / n.norm()
    return n

# --- Aides d'affichage ---
def show_pair(title, expr, ndigits=6):
    print(f"\n{title} (exact):")
    display(expr)
    print(f"{title} (numérique, ~1e-{ndigits}):")
    display(expr.evalf(ndigits))


## Formalisme 6x6 "tout rotations"
M = [[R_ext, E_ext],[E_int, R_int]], S = [[SXX, SXY],[SYX, SYY]], tous blocs 3x3 ∈ SO(3).

Interprétation :
- X : triade spatiale, Y : triade temporelle (temps 3D).
- R_ext : rotation d'espace, R_int : rotation d'horloge (temps).
- E_ext : couplage temps→espace (d'où naît la "translation" lue).
- E_int : couplage espace→temps (rétroaction sur l'horloge).

On reste en "tout rotations", donc pas de colonne affine.

In [2]:
def build_M(Rext, Rint, Eext, Eint):
    return sp.Matrix.vstack(sp.Matrix.hstack(Rext, Eext),
                            sp.Matrix.hstack(Eint, Rint))

def build_S(SXX, SXY, SYX, SYY):
    return sp.Matrix.vstack(sp.Matrix.hstack(SXX, SXY),
                            sp.Matrix.hstack(SYX, SYY))

# -- Paramètres d'exemple (modifiables) : angles rationnels de pi --
R_ext = euler_R(sp.pi/7,   sp.pi/6,   sp.pi/5,   "ZYX")
R_int = euler_R(sp.pi/10,  sp.pi/9,  -sp.pi/8,   "ZYX")
E_ext = euler_R(sp.pi/13, -sp.pi/12,  sp.pi/11,  "ZYX")
E_int = euler_R(-sp.pi/16, sp.pi/15,  sp.pi/14,  "ZYX")

SXX = euler_R(sp.pi/9,   -sp.pi/10, sp.pi/18,  "ZYX")
SXY = euler_R(-sp.pi/12,  sp.pi/11, sp.pi/8,   "ZYX")
SYX = euler_R(sp.pi/15,   sp.pi/7, -sp.pi/13,  "ZYX")
SYY = euler_R(sp.pi/10,   sp.pi/9, -sp.pi/8,   "ZYX")

print("Vérification SO(3) des 8 blocs:")
for name, R in [("R_ext",R_ext),("R_int",R_int),("E_ext",E_ext),("E_int",E_int),
                ("SXX",SXX),("SXY",SXY),("SYX",SYX),("SYY",SYY)]:
    print(f"  {name} in SO(3) ? -> {ok_SO3(R)}")

M = build_M(R_ext, R_int, E_ext, E_int)
S = build_S(SXX, SXY, SYX, SYY)

show_pair("M", M)
show_pair("S", S)

Vérification SO(3) des 8 blocs:
  R_ext in SO(3) ? -> True
  R_int in SO(3) ? -> True
  E_ext in SO(3) ? -> True
  E_int in SO(3) ? -> True
  SXX in SO(3) ? -> True
  SXY in SO(3) ? -> True
  SYX in SO(3) ? -> True
  SYY in SO(3) ? -> True

M (exact):


⎡               ⎛1   √5⎞                                                       ↪
⎢            √3⋅⎜─ + ──⎟                                 ________              ↪
⎢               ⎝4   4 ⎠                                ╱ 5   √5     ⎛π⎞   ⎛1  ↪
⎢            ───────────                           -   ╱  ─ - ── ⋅cos⎜─⎟ + ⎜─  ↪
⎢                 2                                  ╲╱   8   8      ⎝7⎠   ⎝8  ↪
⎢                                                                              ↪
⎢                 ________                              ________               ↪
⎢                ╱ 5   √5                              ╱ 5   √5     ⎛π⎞        ↪
⎢          √3⋅  ╱  ─ - ──                             ╱  ─ - ── ⋅sin⎜─⎟        ↪
⎢             ╲╱   8   8                            ╲╱   8   8      ⎝7⎠   ⎛1   ↪
⎢          ───────────────                          ─────────────────── + ⎜─ + ↪
⎢                 2                                          2            ⎝4   ↪
⎢                           

M (numérique, ~1e-6):


⎡0.700629   -0.354067   0.61948    0.926799   -0.332976  -0.173696⎤
⎢                                                                 ⎥
⎢0.509037   0.856414   -0.0862312  0.272133   0.914161   -0.300421⎥
⎢                                                                 ⎥
⎢  -0.5     0.375754    0.780262   0.258819   0.231161   0.937858 ⎥
⎢                                                                 ⎥
⎢0.953623   -0.25779    0.155392   0.868163   0.461598   0.182264 ⎥
⎢                                                                 ⎥
⎢0.217658   0.947169    0.235575   -0.359605  0.838216   -0.409974⎥
⎢                                                                 ⎥
⎣-0.207912  -0.190827   0.959353   -0.34202   0.290381   0.893701 ⎦


S (exact):


⎡    ________                                                                  ↪
⎢   ╱ √5   5     ⎛π ⎞                            ⎛π ⎞    ⎛π⎞   ⎛1   √5⎞    ⎛π⎞ ↪
⎢  ╱  ── + ─ ⋅cos⎜──⎟                       - sin⎜──⎟⋅cos⎜─⎟ + ⎜─ - ──⎟⋅sin⎜─⎟ ↪
⎢╲╱   8    8     ⎝18⎠                            ⎝18⎠    ⎝9⎠   ⎝4   4 ⎠    ⎝9⎠ ↪
⎢                                                                              ↪
⎢    ________                                                                  ↪
⎢   ╱ √5   5     ⎛π ⎞                        ⎛1   √5⎞    ⎛π ⎞    ⎛π⎞      ⎛π ⎞ ↪
⎢  ╱  ── + ─ ⋅sin⎜──⎟                        ⎜─ - ──⎟⋅sin⎜──⎟⋅sin⎜─⎟ + cos⎜──⎟ ↪
⎢╲╱   8    8     ⎝18⎠                        ⎝4   4 ⎠    ⎝18⎠    ⎝9⎠      ⎝18⎠ ↪
⎢                                                                              ↪
⎢                                                          ________            ↪
⎢        1   √5                                           ╱ √5   5     ⎛π⎞     ↪
⎢      - ─ + ──             

S (numérique, ~1e-6):


⎡0.936608   -0.26726  -0.226578  0.886456   -0.437011  0.152372 ⎤
⎢                                                               ⎥
⎢0.165149   0.907064  -0.387248  0.367182   0.864495   0.343258 ⎥
⎢                                                               ⎥
⎢0.309017   0.32528   0.893701   -0.281733  -0.248335  0.926799 ⎥
⎢                                                               ⎥
⎢0.874788   0.321674  0.362313   0.868163   0.461598   0.182264 ⎥
⎢                                                               ⎥
⎢-0.215616  0.928136  -0.303436  -0.359605  0.838216   -0.409974⎥
⎢                                                               ⎥
⎣-0.433884  0.187322  0.881281   -0.34202   0.290381   0.893701 ⎦

## Sondes, préparation, action et lecture

- Base canonique: e1, e2, e3.
- Sonde droite (préparation) f_{i,j} = [e_i ; e_j] (6x1).
- Sonde gauche scalaire l_{p,q}^T = [e_p^T  e_q^T] (1x6).
- Préparation: V0 = S f = [X_eff ; Y_eff].
- Action: V1 = M V0 = [X' ; Y'].

**Lecture espace-temps :**
- Vielbeins e_s (espace), e_t (temps) pour transformer vers "monde".
- Axe d’horloge lu u_t (unitaire).
- Δx = e_s (R_ext X_eff + E_ext Y_eff).
- Δτ_vec = e_t (E_int X_eff + R_int Y_eff), Δτ = u_t^T Δτ_vec.
- v = ||Δx|| / Δτ.

In [3]:
# Base canonique
e1, e2, e3 = sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])
basis = [e1, e2, e3]

def right_probe_pair(i, j):
    """ f_{i,j} = [e_i ; e_j] """
    return sp.Matrix.vstack(basis[i-1], basis[j-1])

def left_scalar_probe(p, q):
    """ l_{p,q}^T = [e_p^T  e_q^T] """
    return sp.Matrix.hstack(basis[p-1].T, basis[q-1].T)

# Choix des sondes (modifiable)
i, j = 1, 3   # f = (e1, e3)
p, q = 2, 2   # l lit X'[2] + Y'[2]
f  = right_probe_pair(i, j)
lT = left_scalar_probe(p, q)

print("f_{i,j} ="); display(f)
print("l_{p,q}^T ="); display(lT)

# Préparation puis action
V0 = S * f                 # [X_eff ; Y_eff]
V1 = M * V0                # [X' ; Y']
show_pair("V0 = S * f", V0)
show_pair("V1 = M * V0", V1)

# Lecture espace-temps (tout rotations)
# Vielbeins: e_s pour espace, e_t pour temps. u_t = axe d'horloge lu.
c = sp.Symbol('c', positive=True)  # vitesse-limite (constante)
e_s = sp.eye(3)                    # vielbein espace (monde)
e_t = (1/c) * sp.eye(3)            # vielbein temps (injecte 1/c)
u_t = sp.Matrix([0, 0, 1])         # on lit la 3e composante temporelle

X_eff = SXX * basis[i-1] + SXY * basis[j-1]
Y_eff = SYX * basis[i-1] + SYY * basis[j-1]

Delta_x     = e_s * (R_ext*X_eff + E_ext*Y_eff)
Delta_tau_v = e_t * (E_int*X_eff + R_int*Y_eff)
Delta_tau   = sp.simplify((u_t.T * Delta_tau_v)[0])
v_read      = sp.simplify(Delta_x.norm() / Delta_tau)

show_pair("Delta_x (monde)", Delta_x)
print("\nDelta_tau (exact):"); display(Delta_tau)
print("Delta_tau (numérique, c=1):"); display(Delta_tau.subs({c:1}).evalf(6))
print("\nVitesse lue v = ||Delta_x||/Delta_tau (exact):"); display(v_read)
print("v (numérique, c=1):"); display(v_read.subs({c:1}).evalf(6))

# Décomposition "8 termes" lisible via pandas (labels texte + objets sympy)
termXX_p = sp.simplify((R_ext * SXX)[p-1, i-1])
termXY_p = sp.simplify((R_ext * SXY)[p-1, j-1])
termYX_p = sp.simplify((E_ext * SYX)[p-1, i-1])
termYY_p = sp.simplify((E_ext * SYY)[p-1, j-1])

termXX_q = sp.simplify((E_int * SXX)[q-1, i-1])
termXY_q = sp.simplify((E_int * SXY)[q-1, j-1])
termYX_q = sp.simplify((R_int * SYX)[q-1, i-1])
termYY_q = sp.simplify((R_int * SYY)[q-1, j-1])

df8 = pd.DataFrame([
    ("(R_ext SXX)_{p,i}", termXX_p, sp.N(termXX_p, 6)),
    ("(R_ext SXY)_{p,j}", termXY_p, sp.N(termXY_p, 6)),
    ("(E_ext SYX)_{p,i}", termYX_p, sp.N(termYX_p, 6)),
    ("(E_ext SYY)_{p,j}", termYY_p, sp.N(termYY_p, 6)),
    ("(E_int SXX)_{q,i}", termXX_q, sp.N(termXX_q, 6)),
    ("(E_int SXY)_{q,j}", termXY_q, sp.N(termXY_q, 6)),
    ("(R_int SYX)_{q,i}", termYX_q, sp.N(termYX_q, 6)),
    ("(R_int SYY)_{q,j}", termYY_q, sp.N(termYY_q, 6)),
], columns=["terme", "exact", "num ~1e-6"])
print("\nContributions (8 termes) :")
display(df8)

f_{i,j} =


⎡1⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣1⎦

l_{p,q}^T =


[0  1  0  0  1  0]


V0 = S * f (exact):


...


V1 = M * V0 (exact):


⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                  ⎛      ________             ↪
⎢                                                  ⎜     ╱ 5   √5     ⎛π⎞   ⎛1 ↪
⎢                                                  ⎜-   ╱  ─ - ── ⋅cos⎜─⎟ + ⎜─ ↪
⎢                                                  ⎝  ╲╱   8   8      ⎝7⎠   ⎝8 ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                ⎛⎛  √6   √2⎞    ⎛π ⎞    ⎛π ⎞  ↪
⎢                                                ⎜⎜- ── + ──⎟⋅sin⎜──⎟⋅sin⎜──⎟  ↪
⎢                           

V1 = M * V0 (numérique, ~1e-6):


⎡ 2.45664 ⎤
⎢         ⎥
⎢0.460803 ⎥
⎢         ⎥
⎢ 1.17102 ⎥
⎢         ⎥
⎢ 1.81218 ⎥
⎢         ⎥
⎢-0.083313⎥
⎢         ⎥
⎣ 0.7299  ⎦

KeyboardInterrupt: 

## Design d'un "pas de translation" purement rotationnel et vitesse bornée par c

But : choisir SYX, SYY ∈ SO(3) pour produire une cible t_world* = Delta_t * v * d_hat,
sans gain d'amplitude (tout rotations), et avec v ≤ c.

Idée :
- On oriente deux vecteurs a, b dans l'espace local (lié à E_ext) de sorte que
  Y_eff = a + b aligne la translation lue Delta_x = e_s (E_ext Y_eff) avec d_hat,
  tout en contrôlant l'amplitude via un angle theta (r = 2 cos(theta)).
- On construit SYX : e_i -> a et SYY : e_j -> b (rotations minimales).
- On obtient alors le triplet (Delta_x, Delta_tau, v) par la même lecture que précédemment.

In [ ]:
def design_step_SO3(e_s, e_t, E_ext, i, j, t_world_star):
    """
    Conçoit (SYX, SYY) ∈ SO(3) pour réaliser la cible spatiale t_world_star (monde)
    en un pas, par rotations uniquement.
    Hyp: ||t_world_star|| = r <= 2 (si e_s = I, E_ext = I) -> prendre Delta_t = 2/c pour v<=c.
    """
    # Passage au repère local où "E_ext" agit naturellement
    # Ici, e_s et e_t peuvent différer mais on utilise E_ext côté espace.
    r = t_world_star.norm()
    if r == 0:
        return I3, I3, sp.zeros(3,1)
    d_hat_world = t_world_star / r
    # On descend dans le "local espace" de E_ext
    # Inversion locale par pseudo-inverse (ici, E_ext ∈ SO(3) donc E_ext.T)
    d_local = E_ext.T * d_hat_world
    d_local = d_local / d_local.norm()
    # Paramètre d'amplitude via angle theta: r = 2 cos(theta) (borne r<=2)
    theta = sp.acos(sp.Min(1, sp.Max(-1, r/2)))  # sûreté numérique
    n = any_perp(d_local)
    a =  sp.cos(theta)*d_local + sp.sin(theta)*n
    b =  sp.cos(theta)*d_local - sp.sin(theta)*n

    # Rotations qui envoient e_i -> a et e_j -> b
    ei, ej = [sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])][i-1], \
             [sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])][j-1]
    SYX = rotation_from_two_vectors(ei, a)
    SYY = rotation_from_two_vectors(ej, b)

    # Translation lue (espace du monde)
    Y_eff = a + b
    Delta_x_world = e_s * (E_ext * Y_eff)
    return SYX, SYY, Delta_x_world

def one_step(e_s, e_t, R_ext, R_int, E_ext, E_int, SXX, SXY, SYX, SYY, i, j, u_t, c_val=1):
    M_loc = build_M(R_ext, R_int, E_ext, E_int)
    S_loc = build_S(SXX, SXY, SYX, SYY)
    f_loc = sp.Matrix.vstack([sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])][i-1],
                             [sp.Matrix([1,0,0]), sp.Matrix([0,1,0]), sp.Matrix([0,0,1])][j-1])
    V0_loc = S_loc * f_loc
    V1_loc = M_loc * V0_loc
    X_eff = V0_loc[:3,:]
    Y_eff = V0_loc[3:,:]

    Delta_x   = e_s * (R_ext*X_eff + E_ext*Y_eff)
    Delta_tau_vec = e_t.subs({c: c_val}) * (E_int*X_eff + R_int*Y_eff)
    Delta_tau = (u_t.T * Delta_tau_vec)[0]
    return sp.simplify(Delta_x), sp.simplify(Delta_tau), V0_loc, V1_loc, S_loc, M_loc

# --- Démo multipas (exemple) ---
# Vielbeins (monde non-euclidien) : choisis ici e_s = I, e_t = (1/c) I
e_s_demo = sp.eye(3)
e_t_demo = (1/c) * sp.eye(3)

# On réutilise R_ext, R_int, E_ext, E_int, SXX, SXY définis plus haut
i, j = 1, 3
u_t_demo = sp.Matrix([0,0,1])

# Calibration relativiste: c = 1 -> Delta_t = 2
c_val = 1
Delta_t = 2 / c_val

# Trois pas successifs avec cibles t* = Delta_t * v * d_hat
targets = [
    (sp.Matrix([1,0,0]), 1.0),
    (sp.Matrix([0,1,0]), 0.5),
    (sp.Matrix([0,0,1]), 1.0),
]

X_acc = sp.zeros(3,1)
print(f"Delta_t = {Delta_t} (c = {c_val})")
for k, (d_hat, v_req) in enumerate(targets, start=1):
    t_star = Delta_t * v_req * d_hat
    SYX_k, SYY_k, Dx_design = design_step_SO3(e_s_demo, e_t_demo, E_ext, i, j, t_star)
    Dx, Dt, V0_k, V1_k, S_blk, M_blk = one_step(
        e_s_demo, e_t_demo, R_ext, R_int, E_ext, E_int, SXX, SXY, SYX_k, SYY_k, i, j, u_t_demo, c_val
    )
    speed_k = (Dx.norm() / Dt).subs({c: c_val})
    X_acc += Dx
    print(f"\n-- Pas {k} --")
    print("t* (monde) cible :"); display(t_star.evalf(6))
    print("Delta_x réalisé :"); display(Dx.evalf(6))
    print("Delta_tau réalisé :"); display(Dt.evalf(6))
    print("vitesse par pas = ||Delta_x|| / Delta_tau :"); display(sp.N(speed_k, 6))

print("\nPosition monde accumulée (somme des Delta_x) :")
display(X_acc.evalf(6))

## Discussion et options avancées

1) Pourquoi tout en rotations ?
   - Met sur un pied d’égalité rotation et "translation" (via E_ext).
   - Évite les colonnes affines, garde la symétrie bloc 3x3.

2) Borne de vitesse et c :
   - Introduite par le vielbein temps e_t = (1/c) I et par le design "r = 2 cos(theta)".
   - En fixant Delta_t = 2/c (c=1 => Delta_t=2), on obtient v <= c par pas, sans gains.

3) Temps "3D" :
   - Y représente l’horloge 3D (orientation et dynamique).
   - La "mesure" d’un scalaire de temps Δτ s’obtient par projection sur u_t (sonde).

4) Invariance forte type Minkowski (optionnel) :
   - Si tu veux imposer une pseudo-norme invariante (||Δx||^2 - c^2||Δτ_vec||^2),
     l’espace naturel est SO(3,3) (ou sa version spinorielle en algèbre de Clifford).
   - Ce cours montre qu’on peut rester "tout rotations" + vielbeins + sondes,
     ce qui suffit pour expérimenter et raisonner sur la dualité rotation/translation
     et la lecture du temps 3D.

5) Conseils d’expérimentation :
   - Change (i,j) et (p,q) pour explorer différentes composantes et lectures.
   - Ajuste les angles d’Euler pour M et S ; mets-en certains à 0 pour isoler des effets.
   - Fixe c ≠ 1 pour voir l’effet d’échelles temporelles différentes (substitutions avec .subs({c:...})).